In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [2]:
pd.set_option('display.max_rows', None)

### Affinity

In [21]:
#Affinity credit card data
#Spend_all data represents the % change compared to Jan 2020
df_af = pd.read_csv('data/Affinity - National - Daily.csv')
df_af.head(20)

,year,month,day,freq,spend_all,spend_aap,spend_acf,spend_aer,spend_apg,spend_durables,...,spend_s_all_q1,spend_s_all_q2,spend_s_all_q3,spend_s_all_q4,spend_s_inperson,spend_s_retail_no_grocery,spend_s_retail_w_grocery,spend_19_retail_no_grocery,spend_19_retail_w_grocery,provisional
0,2018,12,31,d,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,0
1,2019,1,1,d,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,0
2,2019,1,2,d,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,0
3,2019,1,3,d,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,0
4,2019,1,4,d,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,0
5,2019,1,5,d,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,0
6,2019,1,6,d,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,0
7,2019,1,7,d,.,.,.,.,.,.,...,.,.,.,.,.,.,.,-.02,-.00246,0
8,2019,1,8,d,.,.,.,.,.,.,...,.,.,.,.,.,.,.,-.0675,-.0425,0
9,2019,1,9,d,.,.,.,.,.,.,...,.,.,.,.,.,.,.,-.117,-.0876,0


In [27]:
#Choosing the columns we need
df_af_adj = df_af[['year', 'month', 'day', 'freq', 'spend_all']]
df_af = df_af[df_af['spend_all'] != '.']
df_af_adj.head(20)

,year,month,day,freq,spend_all
378,2020,1,13,d,-.0155
379,2020,1,14,d,-.0056
380,2020,1,15,d,-.00299
381,2020,1,16,d,-.00941
382,2020,1,17,d,-.0162
383,2020,1,18,d,-.00474
384,2020,1,19,d,.00607
385,2020,1,20,d,.0193
386,2020,1,21,d,.0378
387,2020,1,22,d,.0325


In [32]:
# Convert object column to float
df_af_adj['spend_all'] = df_af_adj['spend_all'].astype(float)

In [35]:
df_af_adj.tail()

,year,month,day,freq,spend_all
1317,2023,9,3,w,0.226
1318,2023,9,10,w,0.205
1319,2023,9,17,w,0.210
1320,2023,9,24,w,0.207
1321,2023,10,1,w,0.191


### Lightcast

In [37]:
#Lightcast job postings data
df_jp = pd.read_csv('data/Job Postings - National - Weekly.csv')
df_jp.head(20) 

,year,month,day_endofweek,bg_posts,bg_posts_ss30,bg_posts_ss55,bg_posts_ss60,bg_posts_ss65,bg_posts_ss70,bg_posts_jz1,bg_posts_jzgrp12,bg_posts_jz2,bg_posts_jz3,bg_posts_jzgrp345,bg_posts_jz4,bg_posts_jz5
0,2020,1,10,-0.0378,-0.0574,-0.05110,-0.08910,-0.0849,0.0165,0.0698,0.00584,-0.00112,-0.05120,-0.06390,-0.06510,-0.09240
1,2020,1,17,0.1160,0.1270,0.08500,0.15100,0.1600,0.1150,0.0826,0.10800,0.11000,0.11300,0.12500,0.11200,0.23200
2,2020,1,24,-0.0954,-0.0957,-0.07080,-0.14500,-0.1140,-0.0110,-0.0684,-0.05460,-0.05310,-0.10900,-0.11600,-0.10900,-0.17900
3,2020,1,31,0.0175,0.0260,0.03690,0.08260,0.0395,-0.1200,-0.0840,-0.05890,-0.05610,0.04720,0.05530,0.06220,0.03950
4,2020,2,7,-0.0945,-0.1000,-0.01570,-0.14700,-0.0802,-0.1130,-0.0410,-0.09070,-0.09610,-0.10700,-0.09990,-0.09250,-0.12000
5,2020,2,14,0.2050,0.2690,0.20400,0.14000,0.2080,0.0648,0.1470,0.16900,0.17200,0.21000,0.21700,0.21600,0.23700
6,2020,2,21,0.0589,0.0940,-0.02590,0.00357,0.0885,-0.1400,-0.0251,0.02650,0.03210,0.07270,0.07380,0.07280,0.08240
7,2020,2,28,0.0766,0.0743,0.14300,0.07420,0.0972,-0.0161,0.0170,0.00839,0.00745,0.03690,0.10400,0.10400,0.29600
8,2020,3,6,0.0243,0.0181,-0.00266,-0.05640,0.0332,0.0211,0.1290,0.05160,0.04310,-0.00371,0.00573,0.00711,0.02440
9,2020,3,13,0.0231,0.0393,0.06920,-0.00477,0.0201,-0.0270,0.0895,0.00970,0.00101,0.00640,0.02450,0.03860,-0.00405


In [41]:
#Choosing the columns we need
df_jp_adj = df_jp[['year', 'month', 'bg_posts']]
df_jp_adj.head(20)

,year,month,bg_posts
0,2020,1,-0.0378
1,2020,1,0.1160
2,2020,1,-0.0954
3,2020,1,0.0175
4,2020,2,-0.0945
5,2020,2,0.2050
6,2020,2,0.0589
7,2020,2,0.0766
8,2020,3,0.0243
9,2020,3,0.0231


### Paychex&Intuit

In [43]:
#Paychex and Intuit employemnt data
df_emp = pd.read_csv('data/Employment - National - Weekly.csv')
df_emp.head(20)

,year,month,day_endofweek,emp,emp_incq1,emp_incq2,emp_incq3,emp_incq4,emp_incmiddle,emp_incbelowmed,...,emp_subset_reweighted_q3,emp_retail,emp_s72,emp_retail_inclow,emp_retail_incmiddle,emp_retail_inchigh,emp_ss40,emp_ss60,emp_ss65,emp_ss70
0,2020,1,17,0.000975,0.00174,-0.000068,0.002860,-.000529,0.00132,0.000591,...,0.002700,0.000367,0.000288,0.00204,0.000785,-.00443,0.000786,0.001490,0.000912,0.00045
1,2020,1,24,0.000316,-0.00143,0.002140,0.001580,-.00238,0.00187,0.000838,...,0.001470,-0.000890,0.002150,-0.00375,0.002400,-.00976,-0.000203,-0.000654,0.001300,0.00203
2,2020,1,31,-0.000242,-0.00412,0.003160,-0.000188,-.00194,0.00158,0.000507,...,-0.000252,-0.002110,0.001150,-0.00912,0.001370,-.00433,-0.001410,-0.001170,0.003050,0.00161
3,2020,2,7,0.000308,-0.00742,0.003760,0.000769,.0011,0.00235,-0.000317,...,0.000893,-0.003110,0.000731,-0.01380,0.000701,-.000159,-0.001690,0.000595,0.005530,0.00182
4,2020,2,14,0.001300,-0.01020,0.001780,0.004630,.00563,0.00313,-0.002580,...,0.004590,-0.002720,0.001610,-0.01260,0.001360,-.00243,-0.001830,0.002700,0.006770,0.00298
5,2020,2,21,0.002630,-0.01420,-0.000582,0.010400,.0108,0.00463,-0.005540,...,0.009940,-0.000517,0.002900,-0.01600,0.005370,.0023,-0.001380,0.003500,0.007650,0.00454
6,2020,2,28,0.002320,-0.01900,-0.001450,0.012200,.0122,0.00500,-0.007840,...,0.011700,-0.000883,0.001580,-0.02570,0.006990,.0103,-0.002520,0.003010,0.006990,0.00385
7,2020,3,6,-0.000683,-0.02260,-0.003600,0.009810,.00774,0.00273,-0.010500,...,0.010200,-0.005090,-0.008280,-0.03290,0.003750,.0074,-0.005500,0.001460,0.003010,-0.00440
8,2020,3,13,-0.016800,-0.04380,-0.018000,-0.005650,-.00757,-0.01220,-0.027400,...,-0.001650,-0.022900,-0.063400,-0.05210,-0.012200,-.0156,-0.019100,-0.008690,-0.013100,-0.05480
9,2020,3,20,-0.066500,-0.11500,-0.069700,-0.049900,-.0441,-0.06040,-0.086200,...,-0.038000,-0.077500,-0.215000,-0.12200,-0.061900,-.064,-0.061100,-0.038700,-0.060900,-0.19900


In [44]:
#Choosing the columns we need
df_emp_adj = df_emp[['year', 'month', 'emp']]
df_emp_adj.head(20)

,year,month,emp
0,2020,1,0.000975
1,2020,1,0.000316
2,2020,1,-0.000242
3,2020,2,0.000308
4,2020,2,0.001300
5,2020,2,0.002630
6,2020,2,0.002320
7,2020,3,-0.000683
8,2020,3,-0.016800
9,2020,3,-0.066500


### Womply

In [45]:
#Womply small business revenue and open data
df_sbiz = pd.read_csv('data/Womply - National - Weekly.csv')
df_sbiz.head(20)

,year,month,day_endofweek,merchants_all,revenue_all,merchants_professional,revenue_professional,merchants_health,revenue_health,merchants_food_accommodation,...,merchants_other_services,revenue_other_services,merchants_retail,revenue_retail,revenue_inclow,merchants_inclow,revenue_incmiddle,merchants_incmiddle,revenue_inchigh,merchants_inchigh
0,2020,1,12,0.00934,-0.005230,0.004080,-0.011100,0.00811,-0.005080,0.00998,...,0.00902,0.009850,0.01120,-0.011900,-0.005310,0.010500,0.007290,0.011800,0.004880,0.01060
1,2020,1,19,-0.00367,0.000584,-0.004330,-0.000985,-0.00287,-0.016700,-0.00220,...,-0.00380,0.000038,-0.00553,-0.013900,0.015900,-0.002260,-0.010600,-0.003320,-0.000157,-0.00172
2,2020,1,26,-0.00330,0.013900,0.000051,0.017900,-0.00288,0.023000,-0.00435,...,-0.00341,0.001940,-0.00323,0.019600,0.026800,-0.001270,0.018200,-0.002650,0.011300,-0.00213
3,2020,2,2,-0.00259,-0.001570,0.000253,0.002670,-0.00262,0.004640,-0.00369,...,-0.00234,-0.007390,-0.00305,0.009480,0.002370,-0.000103,0.000269,-0.000814,-0.000271,-0.00271
4,2020,2,9,-0.00444,-0.002180,-0.003930,0.010900,-0.00329,0.007270,-0.00351,...,-0.00557,-0.010200,-0.00485,-0.007210,0.000047,-0.002570,0.008100,-0.002160,0.001460,-0.00289
5,2020,2,16,-0.00514,-0.009290,-0.005060,0.006000,-0.00357,-0.011000,-0.00364,...,-0.00664,-0.007900,-0.00480,-0.000171,0.003530,-0.002100,0.003690,-0.003260,-0.010200,-0.00313
6,2020,2,23,-0.00403,-0.009600,-0.006570,-0.005610,-0.00538,0.000535,-0.00546,...,-0.00283,0.004810,-0.00362,-0.016000,-0.011300,-0.001970,-0.006290,-0.001910,0.002310,-0.00219
7,2020,3,1,-0.01150,-0.045700,-0.014600,-0.059000,-0.01390,-0.055000,-0.00965,...,-0.01240,-0.045300,-0.00956,0.000797,-0.027800,-0.006530,-0.019900,-0.009560,-0.057500,-0.01290
8,2020,3,8,-0.02090,-0.014500,-0.015400,-0.000047,-0.02260,-0.008180,-0.02440,...,-0.02230,-0.012400,-0.01810,0.007800,-0.010900,-0.013900,-0.006210,-0.019000,-0.014900,-0.02240
9,2020,3,15,-0.02820,-0.129000,-0.025300,-0.046500,-0.02520,-0.080400,-0.02910,...,-0.03090,-0.167000,-0.02580,-0.037500,-0.110000,-0.020000,-0.103000,-0.024900,-0.150000,-0.03030


In [46]:
#Choosing the columns we need
df_sbiz_adj = df_sbiz[['year', 'month', 'merchants_all', 'revenue_all']]
df_sbiz_adj.head(20)

,year,month,merchants_all,revenue_all
0,2020,1,0.00934,-0.005230
1,2020,1,-0.00367,0.000584
2,2020,1,-0.00330,0.013900
3,2020,2,-0.00259,-0.001570
4,2020,2,-0.00444,-0.002180
5,2020,2,-0.00514,-0.009290
6,2020,2,-0.00403,-0.009600
7,2020,3,-0.01150,-0.045700
8,2020,3,-0.02090,-0.014500
9,2020,3,-0.02820,-0.129000


### Department of labor

In [15]:
#Unemployment cliams data
df_uemc = pd.read_csv('data/UI Claims - National - Weekly.csv')
df_uemc.head(20)

,year,month,day_endofweek,initclaims_count_regular,contclaims_count_regular,initclaims_count_pua,contclaims_count_peuc,contclaims_count_pua,initclaims_count_combined,contclaims_count_combined,initclaims_rate_regular,contclaims_rate_regular,initclaims_rate_pua,contclaims_rate_peuc,contclaims_rate_pua,initclaims_rate_combined,contclaims_rate_combined
0,2020,1,4,333862,2124746,0,0,0,333862,2124746,0.205,1.30,0,0,0,.205,1.3
1,2020,1,11,336451,2231589,0,0,0,336451,2231589,0.206,1.37,0,0,0,.206,1.37
2,2020,1,18,277748,2116200,0,0,0,277748,2116200,0.170,1.30,0,0,0,.17,1.3
3,2020,1,25,225390,2055963,0,0,0,225390,2055963,0.138,1.26,0,0,0,.138,1.26
4,2020,2,1,222049,2126523,0,0,0,222049,2126523,0.136,1.30,0,0,0,.136,1.3
5,2020,2,8,217127,2061719,0,0,0,217127,2061719,0.133,1.26,0,0,0,.133,1.26
6,2020,2,15,207101,2074809,0,0,0,207101,2074809,0.127,1.27,0,0,0,.127,1.27
7,2020,2,22,197034,2037237,0,0,0,197034,2037237,0.121,1.25,0,0,0,.121,1.25
8,2020,2,29,214913,2079767,0,0,0,214913,2079767,0.132,1.28,0,0,0,.132,1.28
9,2020,3,7,198167,2033416,0,0,0,198167,2033416,0.122,1.25,0,0,0,.122,1.25


In [47]:
#Choosing the columns we need
df_uemc_adj = df_uemc[['year', 'month', 'initclaims_count_regular', 'contclaims_count_regular']]
df_uemc_adj.head(20)

,year,month,initclaims_count_regular,contclaims_count_regular
0,2020,1,333862,2124746
1,2020,1,336451,2231589
2,2020,1,277748,2116200
3,2020,1,225390,2055963
4,2020,2,222049,2126523
5,2020,2,217127,2061719
6,2020,2,207101,2074809
7,2020,2,197034,2037237
8,2020,2,214913,2079767
9,2020,3,198167,2033416


In [9]:
df_af.describe()

,year,month,day,provisional
count,1322.000000,1322.000000,1322.000000,1322.000000
mean,2020.369138,6.134644,15.680787,0.002269
std,1.118918,3.447896,8.823412,0.047601
min,2018.000000,1.000000,1.000000,0.000000
25%,2019.000000,3.000000,8.000000,0.000000
50%,2020.000000,6.000000,16.000000,0.000000
75%,2021.000000,9.000000,23.000000,0.000000
max,2023.000000,12.000000,31.000000,1.000000


In [3]:
# Example: Load CSV files into a list of DataFrames
file_paths = ['data/Affinity - National - Daily.csv', 
'data/Employment - State - Weekly.csv', 
'data/Job Postings - National - Weekly.csv', 
'data/UI Claims - National - Weekly.csv', 
'data/Womply - National - Weekly.csv'] 

dataframes = [pd.read_csv(file) for file in file_paths]